# api_dev.ipynb

In [32]:
import json
import pandas as pd
import io, os
from utils import db

## Electricity Data

In [33]:
e_data = db.load_electricity_data('output.json')
e_data.head()

## Weather Temperature Data

In [35]:
f = open('temp.json')
data = json.load(f)
f.close()
data

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'type': 'FeatureCollection',
 'features': [{'id': 'https://api.weather.gov/stations/KILN/observations/2023-07-31T18:54:00+00:00',
   'type': 'Feature',
   'geometry': {'type': 'Point', 'coordina

In [41]:
len(data['features'])

1200

In [47]:
for i in range(1200):
    if i%100==0:
        print(data['features'][i]['properties'])

{'@id': 'https://api.weather.gov/stations/KILN/observations/2023-07-31T18:54:00+00:00', '@type': 'wx:ObservationStation', 'elevation': {'unitCode': 'wmoUnit:m', 'value': 326}, 'station': 'https://api.weather.gov/stations/KILN', 'timestamp': '2023-07-31T18:54:00+00:00', 'rawMessage': 'KILN 311854Z AUTO 02010KT 10SM FEW055 26/13 A3014 RMK AO2 SLP197 T02610133', 'textDescription': 'Mostly Clear', 'icon': 'https://api.weather.gov/icons/land/day/few?size=medium', 'presentWeather': [], 'temperature': {'unitCode': 'wmoUnit:degC', 'value': 26.1, 'qualityControl': 'V'}, 'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': 13.3, 'qualityControl': 'V'}, 'windDirection': {'unitCode': 'wmoUnit:degree_(angle)', 'value': 20, 'qualityControl': 'V'}, 'windSpeed': {'unitCode': 'wmoUnit:km_h-1', 'value': 18.36, 'qualityControl': 'V'}, 'windGust': {'unitCode': 'wmoUnit:km_h-1', 'value': None, 'qualityControl': 'Z'}, 'barometricPressure': {'unitCode': 'wmoUnit:Pa', 'value': 102070, 'qualityControl': 'V'}, 'se

In [98]:
df = pd.DataFrame(data['features'])
df.columns

Index(['id', 'type', 'geometry', 'properties'], dtype='object')

In [102]:
df.id[0]

'https://api.weather.gov/stations/KILN/observations/2023-07-31T18:54:00+00:00'

In [87]:
[x['id'] for x in data['features']]

['https://api.weather.gov/stations/KILN/observations/2023-07-31T18:54:00+00:00',
 'https://api.weather.gov/stations/KVTA/observations/2023-07-31T18:54:00+00:00',
 'https://api.weather.gov/stations/KLHQ/observations/2023-07-31T18:53:00+00:00',
 'https://api.weather.gov/stations/KMNN/observations/2023-07-31T18:53:00+00:00',
 'https://api.weather.gov/stations/KOSU/observations/2023-07-31T18:53:00+00:00',
 'https://api.weather.gov/stations/KZZV/observations/2023-07-31T18:53:00+00:00',
 'https://api.weather.gov/stations/KCVG/observations/2023-07-31T18:52:00+00:00',
 'https://api.weather.gov/stations/KMFD/observations/2023-07-31T18:52:00+00:00',
 'https://api.weather.gov/stations/KCAK/observations/2023-07-31T18:51:00+00:00',
 'https://api.weather.gov/stations/KCLE/observations/2023-07-31T18:51:00+00:00',
 'https://api.weather.gov/stations/KCMH/observations/2023-07-31T18:51:00+00:00',
 'https://api.weather.gov/stations/KYNG/observations/2023-07-31T18:51:00+00:00',
 'https://api.weather.gov/st

In [79]:
[(x['temperature']) for x in df.properties]

[{'unitCode': 'wmoUnit:degC', 'value': 26.1, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 26.1, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 25.6, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 25.6, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 26.7, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 25.6, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 26.7, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': None, 'qualityControl': 'Z'},
 {'unitCode': 'wmoUnit:degC', 'value': 24.4, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 25, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 25, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 25, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 24, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 25, 'qualityControl': 'V'},
 {'unitCode': 'wmoUnit:degC', 'value': 25.6,

In [69]:
df = pd.DataFrame(data['features'])
for key,value in df.properties[0].items():
    if key in property_list:
        print(key, value)


elevation {'unitCode': 'wmoUnit:m', 'value': 326}
temperature {'unitCode': 'wmoUnit:degC', 'value': 26.1, 'qualityControl': 'V'}
dewpoint {'unitCode': 'wmoUnit:degC', 'value': 13.3, 'qualityControl': 'V'}
windDirection {'unitCode': 'wmoUnit:degree_(angle)', 'value': 20, 'qualityControl': 'V'}
windSpeed {'unitCode': 'wmoUnit:km_h-1', 'value': 18.36, 'qualityControl': 'V'}
windGust {'unitCode': 'wmoUnit:km_h-1', 'value': None, 'qualityControl': 'Z'}
barometricPressure {'unitCode': 'wmoUnit:Pa', 'value': 102070, 'qualityControl': 'V'}
seaLevelPressure {'unitCode': 'wmoUnit:Pa', 'value': 101970, 'qualityControl': 'V'}
visibility {'unitCode': 'wmoUnit:m', 'value': 16090, 'qualityControl': 'C'}
maxTemperatureLast24Hours {'unitCode': 'wmoUnit:degC', 'value': None}
minTemperatureLast24Hours {'unitCode': 'wmoUnit:degC', 'value': None}
precipitationLastHour {'unitCode': 'wmoUnit:mm', 'value': None, 'qualityControl': 'Z'}
precipitationLast3Hours {'unitCode': 'wmoUnit:mm', 'value': None, 'qualityC

In [59]:
property_list = ['elevation',
'temperature', 
'dewpoint', 
'windDirection', 
'windSpeed', 
'windGust', 
'barometricPressure', 
'seaLevelPressure', 
'visibility', 
'maxTemperatureLast24Hours',
'minTemperatureLast24Hours',
'precipitationLastHour',
'precipitationLast3Hours',
'precipitationLast6Hours',
'relativeHumidity',
'windChill',
'heatIndex',
'cloudLayers'] 
